# Train an IntegratedML model on Marketing Campaign Dataset
## Use JDBC to connect to InterSystems IRIS database
This Notebook demonstrates:
- Using the JayDeBeApi Python library to connect to InterSystems IRIS
- Creating views to segment data into training and test sets
- Defining and training an IntegratedML model to predict marketing campaign responses
- Comparing the resulting model's predictions to data in the test set (that the model was not trained on)
- Using the IntegratedML "VALIDATE MODEL" command to calculate accuracy metrics on the test set data

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

### 1. Set environment variables, if necessary

### 2. Get jdbc connection and cursor

In [2]:

import jaydebeapi
url = "jdbc:IRIS://irisimlsvr:1972/USER"
driver = "com.intersystems.jdbc.IRISDriver"
user = "SUPERUSER"
password = "SYS"
#libx = "C:/InterSystems/IRIS20194/dev/java/lib/JDK18"
#jarfile = "C:/InterSystems/IRIS20194/dev/java/lib/JDK18/intersystems-jdbc-3.0.0.jar"
jarfile = "./intersystems-jdbc-3.1.0.jar"

In [3]:
conn = jaydebeapi.connect(driver, url, [user, password], jarfile)
curs = conn.cursor()

### 3. specify the source data table

In [4]:
dataTable = 'Marketing.Campaign'
trainingTable = "Marketing.CampaignTrainingSmall"
predictTable = "Marketing.CampaignPredictSmall"

### 4. Execute a query and display results in Pandas DataFrame

In [5]:
import pandas as pd
from IPython.display import display

df = pd.read_sql("select TOP 20 * from %s" % dataTable, conn)
display(df)

,AGE,EDUCATION,MARITAL_STATUS,OCCUPATION,RELATIONSHIP,RACE,GENDER,PROSPECT_ID,EMPLOYMENT,YEARS_EDUC,COUNTRY_OF_BIRTH,INV_EARNINGS,INV_LOSSES,WORKWEEK_LENGTH,RESPONSE,AMOUNT
0,50,HIGHSCHOOL,MARRIEDCIVSPO,REPAIRCRAFT,HUSBAND,WHITE,MALE,77001107,BUSINESS,9,USA,0,0,40,0,123.90
1,49,HIGHSCHOOL,MARRIEDCIVSPO,LOGISTICSTRANSP,HUSBAND,WHITE,MALE,77001140,BUSINESS,9,USA,0,0,40,0,97.91
2,37,HIGHSCHOOL,MARRIEDCIVSPO,REPAIRCRAFT,HUSBAND,WHITE,MALE,77001143,BUSINESS,9,USA,0,0,40,0,135.65
3,43,HIGHSCHOOL,MARRIEDCIVSPO,CLEANINGHANDLING,HUSBAND,WHITE,MALE,77001187,BUSINESS,9,USA,0,0,40,0,93.23
4,54,HIGHSCHOOL,MARRIEDCIVSPO,LOGISTICSTRANSP,HUSBAND,WHITE,MALE,77001412,BUSINESS,9,USA,0,0,40,0,143.72
5,27,HIGHSCHOOL,MARRIEDCIVSPO,SERVICEVARIOUS,HUSBAND,WHITE,MALE,77001429,BUSINESS,9,USA,0,0,40,0,232.20
6,29,HIGHSCHOOL,MARRIEDCIVSPO,REPAIRCRAFT,HUSBAND,WHITE,MALE,77001443,BUSINESS,9,USA,0,0,40,0,103.46
7,29,HIGHSCHOOL,MARRIEDCIVSPO,ADMINOFFICE,HUSBAND,WHITE,MALE,77001484,BUSINESS,9,USA,0,0,40,0,123.53
8,21,HIGHSCHOOL,MARRIEDCIVSPO,MACHINEOPERINSP,HUSBAND,WHITE,MALE,77001526,BUSINESS,9,USA,0,0,40,0,230.07
9,36,HIGHSCHOOL,MARRIEDCIVSPO,MACHINEOPERINSP,HUSBAND,WHITE,MALE,77001634,BUSINESS,9,USA,0,0,40,0,48.95


In [6]:
# Show number rows
df1 = pd.read_sql("SELECT COUNT(*) FROM %s" % dataTable, conn)
display(df1)

,Aggregate_1
0,48842


### Cleaning before retrying

In [7]:
# Before executing the notebook again, drop model, tables, views,... previously created
#curs.execute("DROP VIEW %s" % trainingTable)
#curs.execute("DROP VIEW %s" % predictTable)
#curs.execute("DROP MODEL CampaignModel")

### 5. Make some views to split training and testing datasets

In [8]:
# Small training set view
curs.execute("CREATE VIEW %s AS SELECT * FROM %s WHERE ID<3001" % (trainingTable,dataTable))
# Small prediction set
curs.execute("CREATE VIEW %s AS SELECT * FROM %s WHERE ID>47000" % (predictTable,dataTable))

### 6. Create and Train an IntegratedML Model using default settings
IntegratedML only needs a model name, the name of the column that is the target column to predict, and a table (or SELECT query to specify input columns.

In [9]:
curs.execute("CREATE MODEL CampaignModel PREDICTING (RESPONSE) FROM %s" % trainingTable)

Now that the model is defined, you can TRAIN it, which invokes the AutoML machine learning procedure.

In [10]:
curs.execute("TRAIN MODEL CampaignModel")

Once that finishes, you can see some information about the model in the "ML_TRAINED_MODELS" table.

In [11]:
df3 = pd.read_sql("SELECT * FROM INFORMATION_SCHEMA.ML_TRAINED_MODELS", conn)
display(df3)

,MODEL_NAME,TRAINED_MODEL_NAME,PROVIDER,TRAINED_TIMESTAMP,MODEL_TYPE,MODEL_INFO
0,CampaignModel,CampaignModel_t1,AutoML,2022-04-22 08:01:09.859000,classification,"ModelType:TensorFlow Neural Network, Package:T..."
1,bc,bc_t1,AutoML,2022-04-22 07:54:38.592000,classification,"ModelType:TensorFlow Neural Network, Package:T..."


### 7. Compare model output to data it has not seen yet
Now you can use SQL to SELECT data from another table, run the IntegratedML model on this new data, and see how well the predictions match the data!

In [12]:
df4 = pd.read_sql("SELECT PREDICT(CampaignModel) AS PredictedResponse, \
                  response AS ActualResponse FROM %s" % predictTable, conn)
display(df4)

,PredictedResponse,ActualResponse
0,0,0
1,0,0
2,0,1
3,0,0
4,0,1
...,...,...
1837,0,0
1838,0,0
1839,0,0
1840,0,0


### 8. VALIDATE MODEL command calculates accuracy metrics
You can certainly take that output above and calculate the accuracy using a standard formula, but IntegratedML has a built-in function to do that!

Each time you run the command "VALIDATE MODEL..." it generates a set of metrics calculated on the data passed into the query. Since this table can be a bit difficult to read in its raw form we use a simple "pivot" call to arrange the data.

In [13]:
curs.execute("VALIDATE MODEL CampaignModel FROM %s" % predictTable)
df5 = pd.read_sql("SELECT * FROM INFORMATION_SCHEMA.ML_VALIDATION_METRICS", conn)
df6 = df5.pivot(index='VALIDATION_RUN_NAME', columns='METRIC_NAME', values='METRIC_VALUE')
display(df6)

METRIC_NAME,Accuracy,F-Measure,Precision,Recall
VALIDATION_RUN_NAME,,,,
CampaignModel_t1_v1,0.6,0.2,0.81,0.11
